In [1]:
import os
import sys
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one level
sys.path.append(project_path)
import pandas as pd
from numpy.random import random
from utils.chatbot import QWENChatBot
import time
import random
import json
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
data_tagged = pd.read_excel('../processed_data/01_tagged_raw.xlsx')
data_tagged

,内容,raw_tagging
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""..."
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""..."
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""..."
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."


In [7]:

def safe_json_parser(json_string):
    try:
        # Try parsing the JSON
        return json.loads(json_string)
    except json.JSONDecodeError:
        # If an error occurs, skip/return None
        return None

# Apply the function to the column of JSON data
data_tagged['parsed_json'] = data_tagged['raw_tagging'].apply(safe_json_parser)
# Optionally, drop rows where parsing failed (parsed_json is None)
data_tagged = data_tagged.dropna(subset=['parsed_json']).reset_index(drop=True)

In [8]:
#data_tagged['raw_tagging'] = data['raw_tagging'].apply(json.loads)
data_tagged['Age_Range'] = data_tagged['parsed_json'].apply(lambda x: x.get('Age_Range', np.nan) if x.get('Age_Range', '') != '' else np.nan)
data_tagged['Gender'] = data_tagged['parsed_json'].apply(lambda x: x.get('Gender', np.nan) if x.get('Gender', '') != '' else np.nan)
data_tagged['City_Tier'] = data_tagged['parsed_json'].apply(lambda x: x.get('City_Tier', np.nan) if x.get('City_Tier', '') != '' else np.nan)
data_tagged['Purchase_Purpose'] = data_tagged['parsed_json'].apply(lambda x: x.get('Purchase_Purpose', np.nan) if x.get('Purchase_Purpose', '') != '' else np.nan)
data_tagged['Purchase_Situation'] = data_tagged['parsed_json'].apply(lambda x: x.get('Purchase_Situation', np.nan) if x.get('Purchase_Situation', '') != '' else np.nan)
data_tagged['Product_Mention'] = data_tagged['parsed_json'].apply(lambda x: x.get('Product_Mention', np.nan) if x.get('Product_Mention', []) != [] else np.nan)
data_tagged['Competitor'] = data_tagged['parsed_json'].apply(lambda x: x.get('Competitor', np.nan) if x.get('Competitor', []) != [] else np.nan)
data_tagged['Keywords_Design_Elements'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Design_Elements', np.nan) if x.get('Keywords', {}).get('Design_Elements', []) != [] else np.nan)
data_tagged['Keywords_Design_Style'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Design_Style', np.nan) if x.get('Keywords', {}).get('Design_Style', []) != [] else np.nan)
data_tagged['Keywords_Functionality'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Functionality', np.nan) if x.get('Keywords', {}).get('Functionality', []) != [] else np.nan)
data_tagged['Keywords_Other'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Other', np.nan) if x.get('Keywords', {}).get('Other', []) != [] else np.nan)
data_tagged['Emotion'] = data_tagged['parsed_json'].apply(lambda x: x.get('Emotion', np.nan) if x.get('Emotion', '') != '' else np.nan)
data_tagged['Past_Purchase'] = data_tagged['parsed_json'].apply(lambda x: x.get('Past_Purchase', np.nan) if x.get('Past_Purchase', '') != '' else np.nan)
data_tagged['Tone'] = data_tagged['parsed_json'].apply(lambda x: x.get('Tone', np.nan) if x.get('Tone', '') != '' else np.nan)

data_tagged


,内容,raw_tagging,parsed_json,Age_Range,Gender,City_Tier,Purchase_Purpose,Purchase_Situation,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Keywords_Other,Emotion,Past_Purchase,Tone
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[联名快闪店, lv和村上隆]",NaN,否,活泼俏皮
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""...","{'Age_Range': '25-34岁', 'Gender': '女性', 'City_...",25-34岁,女性,一线,个人喜好/收藏,NaN,[LV水貂帽衫背心],"[miumiu, Chanel]",[水貂帽衫背心],NaN,NaN,"[LV村上隆, 北京skp店, vic客户体验]",喜欢,是,成熟稳重
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,打卡/拍照,NaN,NaN,NaN,NaN,"[黑白三彩店, 咖啡店]",NaN,"[快闪店, LV村上隆]",喜欢,否,活泼俏皮
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[黑三彩Carryall, 黑武士羊皮carryall]",NaN,"[黑三彩, 黑武士羊皮]",NaN,"[单肩斜挎, 不压身高不显大]",NaN,喜欢,是,成熟稳重
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n...","{'Age_Range': '', 'Gender': '女性', 'City_Tier':...",NaN,女性,NaN,追求独特设计,NaN,"[卡包, 钱包]",NaN,"[白三彩, 黑三彩]",[可爱],[实用],NaN,喜欢,否,活泼俏皮
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,陪伴购物,逛街,[trainer],NaN,NaN,NaN,NaN,"[母上大人, 蹭到, 心动]",开心,否,活泼俏皮
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[村上隆太阳花熊猫, 手袋]",NaN,"[太阳花, 熊猫]",[合适],NaN,[花花],喜欢,否,活泼俏皮
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[快闪店, 咖啡店, 出片, 跨年, ccd拍照]",满意,否,活泼俏皮
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[speedy系列, speedynano]",NaN,"[speedy30小, 上身偏大]",[酷],NaN,[oversize],满意,否,成熟稳重


In [13]:
data_tagged.isnull().sum()/len(data_tagged)

内容                          0.000000
raw_tagging                 0.000000
parsed_json                 0.000000
Age_Range                   0.728968
Gender                      0.583175
City_Tier                   0.784164
Purchase_Purpose            0.373430
Purchase_Situation          0.635325
Product_Mention             0.264560
Competitor                  0.923106
Keywords_Design_Elements    0.509707
Keywords_Design_Style       0.526456
Keywords_Functionality      0.791778
Keywords_Other              0.075752
Emotion                     0.175866
Past_Purchase               0.131709
Tone                        0.174343
dtype: float64

In [14]:

data_tagged.Purchase_Purpose.value_counts()

Purchase_Purpose
追求独特设计    834
个人喜好       47
追求时尚       28
收藏         28
日常使用       24
个人使用       21
实用需求       17
追求潮流       17
实用性        16
送礼         16
Name: count, dtype: int64

In [15]:
data_tagged.Purchase_Situation.value_counts()[:10]

Purchase_Situation
日常使用    114
日常穿搭     46
日常出街     22
逛街       21
日常搭配     20
出街       17
新年       16
店内购买     15
旅行购物     14
日常       14
Name: count, dtype: int64

In [7]:
data_tagged.columns

Index(['内容', 'raw_tagging', 'parsed_json', 'Age_Range', 'Gender', 'City_Tier',
       'Purchase_Purpose', 'Purchase_Situation', 'Product_Mention',
       'Competitor', 'Keywords_Design_Elements', 'Keywords_Design_Style',
       'Keywords_Functionality', 'Keywords_Other', 'Emotion', 'Past_Purchase',
       'Tone'],
      dtype='object')

In [5]:
cols = ['Age_Range', 'Gender', 'City_Tier',
       'Purchase_Purpose', 'Purchase_Situation', 'Product_Mention',
       'Competitor', 'Keywords_Design_Elements', 'Keywords_Design_Style',
       'Keywords_Functionality', 'Emotion', 'Past_Purchase',
       'Tone']

In [11]:
data_tagged['Product_Mention'].value_counts()

Product_Mention
[白三彩]                                               88
[lv白三彩]                                             30
[lv村上隆]                                             22
[speedy]                                            15
[trainer]                                           12
                                                    ..
[lv村上隆1.0, lv村上隆2.0]                                 1
[speedy, LV村上隆联名, 白三彩]                               1
[LV soft trunk迷你方盒子, Tyler泰勒联名芯片款, 小狗包, LV白三彩系列]     1
[all in bb, 黑三彩nano, 白三彩水桶包]                         1
[speedy系列, speedynano]                               1
Name: count, Length: 1511, dtype: int64

In [12]:
length = 0
for col in cols:
    length+= len(data_tagged[col].value_counts())
length

4909

In [18]:
data_tagged.to_excel('../processed_data/02_tagged_processed.xlsx',index=False)

In [17]:
from collections import Counter
def count_word_freq(column):
    # Flatten all lists from the column into a single list, ignoring NaN values
    all_keywords = [word for keywords in column.dropna() for word in keywords]
    
    # Count the frequency of each word using Counter
    return Counter(all_keywords)
# Function to get top N frequent keywords
def get_top_n(counter_obj, n=5):
    return counter_obj.most_common(n)


# Count word frequencies for each subcategory
freq_product = count_word_freq(data_tagged['Product_Mention'])
freq_competitor = count_word_freq(data_tagged['Competitor'])
freq_design_elements = count_word_freq(data_tagged['Keywords_Design_Elements'])
freq_design_style = count_word_freq(data_tagged['Keywords_Design_Style'])
freq_functionality = count_word_freq(data_tagged['Keywords_Functionality'])
freq_other = count_word_freq(data_tagged['Keywords_Other'])

# Display top 5 keywords for each subcategory
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_product, 10))
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_competitor, 10))
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_design_elements, 10))

print("\nDesign Style Word Frequency (Top 5):")
print(get_top_n(freq_design_style, 10))

print("\nFunctionality Word Frequency (Top 5):")
print(get_top_n(freq_functionality, 10))

print("\nOther Word Frequency (Top 5):")
print(get_top_n(freq_other, 15))

Design Elements Word Frequency (Top 5):
[('白三彩', 322), ('黑三彩', 104), ('lv白三彩', 94), ('lv村上隆', 80), ('speedy', 76), ('carryall', 60), ('麻将包', 36), ('lv村上隆白三彩', 35), ('trainer', 32), ('钱包', 30)]
Design Elements Word Frequency (Top 5):
[('Chanel', 18), ('爱马仕', 18), ('香奈儿', 14), ('miumiu', 12), ('chanel', 12), ('Dior', 9), ('克罗心', 8), ('Prada', 5), ('dior', 5), ('Celine', 4)]
Design Elements Word Frequency (Top 5):
[('白三彩', 310), ('白色', 109), ('黑三彩', 85), ('黑色', 68), ('太阳花', 64), ('三彩', 57), ('蝴蝶结', 44), ('老花', 43), ('彩色', 34), ('黑白三彩', 24)]

Design Style Word Frequency (Top 5):
[('可爱', 212), ('好看', 105), ('经典', 48), ('时尚', 45), ('复古', 39), ('百搭', 30), ('时髦', 30), ('精致', 26), ('美', 25), ('白三彩', 20)]

Functionality Word Frequency (Top 5):
[('实用', 43), ('斜挎', 36), ('容量大', 33), ('百搭', 26), ('能装', 20), ('手拎', 19), ('手提', 18), ('单肩斜挎', 14), ('大容量', 13), ('单肩', 11)]

Other Word Frequency (Top 5):
[('村上隆', 87), ('巨鹿路', 79), ('排队', 77), ('快闪店', 64), ('快闪', 58), ('开箱', 56), ('咖啡', 42), ('新年礼物', 39)